<a href="https://colab.research.google.com/github/adiasija2011/DSA_Python/blob/main/siamesse_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries


Importing the dataset


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_olivetti_faces
from sklearn.utils.validation import check_random_state

In [ ]:
data = fetch_olivetti_faces() #dataset to train and test ton

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


CNN MODEL


In [ ]:

def create_model():
    inputs=keras.Input((64,64,1))
    x = layers.Conv2D(64, (11, 11), padding="same", activation="relu")(inputs)
    x = layers.Conv2D(64, (11, 11), padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, (5, 5), padding="same", activation="relu")(x)
    x = layers.Conv2D(128, (5, 5), padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(x)
    x = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.3)(x)

    pooledOutput = layers.GlobalAveragePooling2D()(x)
    pooledOutput = layers.Dense(1024)(pooledOutput)
    outputs = layers.Dense(128)(pooledOutput)

    model = keras.Model(inputs, outputs)
    return model
    

Obtaining the feature vectors of 2 pictures to be compared by running them through the NN Model

In [ ]:

feature_extractor = create_model()
imgA = keras.Input(shape=(64, 64, 1))
imgB = keras.Input(shape=(64, 64, 1))
featA = feature_extractor(imgA)
featB = feature_extractor(imgB)

In [ ]:
from keras import backend as k

Euclidean distance function for similarity calculation

In [ ]:

def euclidean_distance(vectors):
    (featA, featB) = vectors
    sum_squared = k.sum(k.square(featA - featB), axis=1, keepdims=True)
    return k.sqrt(sum_squared)

distance = layers.Lambda(euclidean_distance)([featA, featB])
outputs = layers.Dense(1, activation="sigmoid")(distance)
model = keras.Model(inputs=[imgA, imgB], outputs=outputs)

In [ ]:
model.compile(loss="BinaryCrossentropy", optimizer="adam" ,metrics=["accuracy"])



function to generate training image pairs

In [ ]:

def generate_train_image_pairs(images_dataset, labels_dataset):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                      [index for index, curr_label in enumerate(labels_dataset) if
                                       label == curr_label])
    
    pair_images = []
    pair_labels = []
    for index, image in enumerate(images_dataset):
        pos_indices = label_wise_indices.get(labels_dataset[index])
        pos_image = images_dataset[np.random.choice(pos_indices)]
        pair_images.append((image, pos_image))
        pair_labels.append(1)

        neg_indices = np.where(labels_dataset != labels_dataset[index])
        neg_image = images_dataset[np.random.choice(neg_indices[0])]
        pair_images.append((image, neg_image))
        pair_labels.append(0)
    return np.array(pair_images), np.array(pair_labels)

In [ ]:
    images_pair, labels_pair = generate_train_image_pairs(images_dataset=data.images, labels_dataset=data.target)
    history = model.fit([images_pair[:, 0], images_pair[:, 1]], labels_pair[:],validation_split=0.1,batch_size=64,epochs=100)
    

Epoch 1/100
12/12 [==============================] - 40s 333ms/step - loss: 0.6950 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 2/100
12/12 [==============================] - 2s 156ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/100
12/12 [==============================] - 2s 156ms/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 4/100
12/12 [==============================] - 2s 156ms/step - loss: 0.6936 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.3750
Epoch 5/100
12/12 [==============================] - 2s 157ms/step - loss: 0.6930 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/100
12/12 [==============================] - 2s 156ms/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/100
12/12 [==============================] - 2s 155ms/step - loss: 0.6936 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.500

function to generate testing image pairs

In [ ]:

def generate_test_image_pairs(images_dataset, labels_dataset, image):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
      label_wise_indices.setdefault(label,[index for index, curr_label in enumerate(labels_dataset) if label == curr_label])
  
    pair_images = []
    pair_labels = []
    for label, indices_for_label in label_wise_indices.items():
        test_image = images_dataset[np.random.choice(indices_for_label)]
        pair_images.append((image, test_image))
        pair_labels.append(label)
    return np.array(pair_images), np.array(pair_labels)

In [ ]:
images_dataset=data.images
labels_dataset=data.target
image = images_dataset[92] # a random image as test image
test_image_pairs, test_label_pairs = generate_test_image_pairs(images_dataset, labels_dataset, image)
 # produce an array of test image pairs and test label pairs
pred=[]
# for each pair in the test image pair, predict the similarity between the images
for index, pair in enumerate(test_image_pairs):
  pair_image1 = np.expand_dims(pair[0], axis=-1)
  pair_image1 = np.expand_dims(pair_image1, axis=0)
  pair_image2 = np.expand_dims(pair[1], axis=-1)
  pair_image2 = np.expand_dims(pair_image2, axis=0)
  prediction = model.predict([pair_image1, pair_image2])[0][0]
  pred.append(prediction)

testing the algorithm on pictures from the internet


In [ ]:

import cv2

In [ ]:
imo1=cv2.imread('imk.jpg')
grayImage1 = cv2.cvtColor(imo1, cv2.COLOR_BGR2GRAY)
grayImage1 = cv2.resize(grayImage1,(64,64))


error: ignored

In [ ]:
imo2=cv2.imread('imk1.jpg')
grayImage2 = cv2.cvtColor(imo2, cv2.COLOR_BGR2GRAY)
grayImage2 = cv2.resize(grayImage2,(64,64)) #resizing the resolution to fit the model

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
cv2_imshow(grayImage2)
cv2_imshow(grayImage1)

In [ ]:
x=np.array((grayImage1,grayImage1))
cv2_imshow(x[0])

In [ ]:
#running sample images through the siamese nn to check for a similarity score
pair1_image1 = np.expand_dims(x[0], axis=-1)
pair1_image1 = np.expand_dims(pair1_image1, axis=0)
pair2_image2 = np.expand_dims(x[1], axis=-1)
pair2_image2 = np.expand_dims(pair2_image2, axis=0)
prediction1 = model.predict([pair1_image1, pair2_image2])[0][0]
print(prediction1)